In [1]:
! pip install jellyfish

In [2]:
import pymysql
import json
import requests
import jellyfish
from datetime import datetime
import re

In [3]:
import pandas
df = pandas.read_csv("../7102.csv")

In [ ]:
timeout = 10
connection = pymysql.connect(
  charset="utf8mb4",
  connect_timeout=timeout,
  cursorclass=pymysql.cursors.DictCursor,
  db="iia_group2",
  host="localhost",
  password="root",
  read_timeout=timeout,
  port=24936,
  user="mint",
  write_timeout=timeout,
)


In [104]:
states = []
cursor = connection.cursor()
cursor.execute("select state from States")
x = cursor.fetchall()
cursor.close()

In [105]:
unique_state_list = [i["state"] for i in x ]

In [ ]:
def check_similarity(col_name_sql,col_name_api):
    pass

def match_state_name(input_state, unique_states_list):
    # Initialize variables to store the best match and highest score
    best_match = None
    highest_score = 0

    # Iterate through the unique states list and calculate Jaro-Winkler scores
    for state in unique_states_list:
        similarity_score = jellyfish.jaro_winkler_similarity(input_state, state)
        if similarity_score > highest_score:
            highest_score = similarity_score
            best_match = state
        # print(similarity_score, state)

    # Return the best matching state name
    if(highest_score < 0.7):
        return None
    return best_match

def get_start_date(string:str):
    try:
        temp = re.split(r"[(),-]",string)
        date_string = "1 " + temp[1] + " " + temp[-1] 
        date_object = datetime.strptime(date_string, "%d %b %Y").date()
        return date_object
    except:
        return None

def get_end_date(string:str):
    try:
        temp = re.split(r"[(),-]",string)
        date_string = "1 " + temp[2] + " " + str(int(temp[-1]) +1)
        date_object = datetime.strptime(date_string, "%d %b %Y").date()
    except:
        return None
    return date_object


In [196]:
def get_month(string: str):
    try:
        temp = re.split(r"[(),-]",string)
        
        date_string = "1 " + temp[-2] + " " + str(int(temp[-1]))
        # print(date_string)
        date_object = datetime.strptime(date_string, "%d %B %Y").date()
    except:
        return None
    return date_object


In [197]:
get_month("December, 2019")

datetime.date(2019, 12, 1)

In [177]:
get_end_date("Financial Year (Apr - Mar), 2017")

datetime.date(2018, 3, 1)

In [40]:
df_per_year = df.drop(["Month","Gst ( goods and service tax ) return type","State lgd code","Country"],axis=1)

In [210]:
# df_per_year
df_per_month = df.drop(["Year","State lgd code","Country"],axis=1)

In [211]:
df_per_month

,State,Month,Gst ( goods and service tax ) return type,Payer eligible for gst ( goods and service tax ) registration_avg,Payer eligible for gst ( goods and service tax ) registration_max,Payer eligible for gst ( goods and service tax ) registration_min,Payer eligible for gst ( goods and service tax ) registration_sum,Payer eligible for gst ( goods and service tax ) registration_count,Payer eligible for gst ( goods and service tax ) registration_stddev,Payer eligible for gst ( goods and service tax ) registration_LandAreaWeight,...,Gst ( goods and service tax ) payers registered after due date_max,Gst ( goods and service tax ) payers registered after due date_min,Gst ( goods and service tax ) payers registered after due date_sum,Gst ( goods and service tax ) payers registered after due date_count,Gst ( goods and service tax ) payers registered after due date_stddev,Gst ( goods and service tax ) payers registered after due date_LandAreaWeight,Gst ( goods and service tax ) payers registered after due date_TotalPopulationWeight,Gst ( goods and service tax ) payers registered after due date_NumberOfHouseholdsWeight,Gst ( goods and service tax ) payers registered after due date_TotalPopulationMaleWeight,Gst ( goods and service tax ) payers registered after due date_TotalPopulationFemaleWeight
0,Chandigarh,"February, 2021",GSTR-3,12710.0,12710.0,12710.0,12710.0,1,NaN,12710.0,...,3628.0,3628.0,3628.0,1,NaN,3628.0,3628.0,3628.0,3628.0,3628.0
1,Sikkim,"December, 2019",GSTR-1,7450.0,7450.0,7450.0,7450.0,1,NaN,7450.0,...,2995.0,2995.0,2995.0,1,NaN,2995.0,2995.0,2995.0,2995.0,2995.0
2,Sikkim,"December, 2019",GSTR-3,7450.0,7450.0,7450.0,7450.0,1,NaN,7450.0,...,2769.0,2769.0,2769.0,1,NaN,2769.0,2769.0,2769.0,2769.0,2769.0
3,Sikkim,"January, 2020",GSTR-1,3908.0,3908.0,3908.0,3908.0,1,NaN,3908.0,...,1605.0,1605.0,1605.0,1,NaN,1605.0,1605.0,1605.0,1605.0,1605.0
4,Jammu And Kashmir,"July, 2017",GSTR-1,52455.0,52455.0,52455.0,52455.0,1,NaN,52455.0,...,7205.0,7205.0,7205.0,1,NaN,7205.0,7205.0,7205.0,7205.0,7205.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4243,The Dadra And Nagar Haveli And Daman And Diu,"March, 2022",GSTR-3,14581.0,14581.0,14581.0,14581.0,1,NaN,14581.0,...,2919.0,2919.0,2919.0,1,NaN,2919.0,2919.0,2919.0,2919.0,2919.0
4244,The Dadra And Nagar Haveli And Daman And Diu,"April, 2022",GSTR-1,11569.0,11569.0,11569.0,11569.0,1,NaN,11569.0,...,2307.0,2307.0,2307.0,1,NaN,2307.0,2307.0,2307.0,2307.0,2307.0
4245,The Dadra And Nagar Haveli And Daman And Diu,"April, 2022",GSTR-3,11569.0,11569.0,11569.0,11569.0,1,NaN,11569.0,...,1441.0,1441.0,1441.0,1,NaN,1441.0,1441.0,1441.0,1441.0,1441.0
4246,The Dadra And Nagar Haveli And Daman And Diu,"May, 2022",GSTR-1,11640.0,11640.0,11640.0,11640.0,1,NaN,11640.0,...,1795.0,1795.0,1795.0,1,NaN,1795.0,1795.0,1795.0,1795.0,1795.0


In [ ]:
# import numpy as np
df_per_year = df_per_year.groupby(["State","Year"]).mean().reset_index()

In [61]:
for i in df_per_year.keys():
    x = i.split('_')
    if(len(x) == 2 and x[1] != "avg"):
        print(df_per_year[i])
        df_per_year = df_per_year.drop([i],axis=1)

In [212]:
for i in df_per_month.keys():
    x = i.split('_')
    if(len(x) == 2 and x[1] != "avg"):
        print(df_per_month[i])
        df_per_month = df_per_month.drop([i],axis=1)

0       12710.0
1        7450.0
2        7450.0
3        3908.0
4       52455.0
         ...   
4243    14581.0
4244    11569.0
4245    11569.0
4246    11640.0
4247    11640.0
Name: Payer eligible for gst ( goods and service tax ) registration_max, Length: 4248, dtype: float64
0       12710.0
1        7450.0
2        7450.0
3        3908.0
4       52455.0
         ...   
4243    14581.0
4244    11569.0
4245    11569.0
4246    11640.0
4247    11640.0
Name: Payer eligible for gst ( goods and service tax ) registration_min, Length: 4248, dtype: float64
0       12710.0
1        7450.0
2        7450.0
3        3908.0
4       52455.0
         ...   
4243    14581.0
4244    11569.0
4245    11569.0
4246    11640.0
4247    11640.0
Name: Payer eligible for gst ( goods and service tax ) registration_sum, Length: 4248, dtype: float64
0       1
1       1
2       1
3       1
4       1
       ..
4243    1
4244    1
4245    1
4246    1
4247    1
Name: Payer eligible for gst ( goods and service tax ) r

In [213]:
df_per_month

,State,Month,Gst ( goods and service tax ) return type,Payer eligible for gst ( goods and service tax ) registration_avg,Gst ( goods and service tax ) payers registered before due date_avg,Gst ( goods and service tax ) payers registered after due date_avg
0,Chandigarh,"February, 2021",GSTR-3,12710.0,9080.0,3628.0
1,Sikkim,"December, 2019",GSTR-1,7450.0,3061.0,2995.0
2,Sikkim,"December, 2019",GSTR-3,7450.0,3717.0,2769.0
3,Sikkim,"January, 2020",GSTR-1,3908.0,1156.0,1605.0
4,Jammu And Kashmir,"July, 2017",GSTR-1,52455.0,38766.0,7205.0
...,...,...,...,...,...,...
4243,The Dadra And Nagar Haveli And Daman And Diu,"March, 2022",GSTR-3,14581.0,10675.0,2919.0
4244,The Dadra And Nagar Haveli And Daman And Diu,"April, 2022",GSTR-1,11569.0,8325.0,2307.0
4245,The Dadra And Nagar Haveli And Daman And Diu,"April, 2022",GSTR-3,11569.0,9131.0,1441.0
4246,The Dadra And Nagar Haveli And Daman And Diu,"May, 2022",GSTR-1,11640.0,8513.0,1795.0


In [214]:
df_per_year

,State,Year,Payer eligible for gst ( goods and service tax ) registration_avg,Gst ( goods and service tax ) payers registered before due date_avg,Gst ( goods and service tax ) payers registered after due date_avg,state,start_date,end_date,registered_payer,eligible_payer
0,Andaman And Nicobar Islands,"Financial Year (Apr - Mar), 2017",3388.833333,1029.555556,1566.944444,andaman and nicobar islands,2017-04-01,2018-03-01,2596.500000,3388.833333
1,Andaman And Nicobar Islands,"Financial Year (Apr - Mar), 2018",4107.000000,926.958333,1945.000000,andaman and nicobar islands,2018-04-01,2019-03-01,2871.958333,4107.000000
2,Andaman And Nicobar Islands,"Financial Year (Apr - Mar), 2019",3570.041667,950.291667,1964.250000,andaman and nicobar islands,2019-04-01,2020-03-01,2914.541667,3570.041667
3,Andaman And Nicobar Islands,"Financial Year (Apr - Mar), 2020",3627.708333,1133.666667,1915.500000,andaman and nicobar islands,2020-04-01,2021-03-01,3049.166667,3627.708333
4,Andaman And Nicobar Islands,"Financial Year (Apr - Mar), 2021",3562.750000,1502.541667,1704.666667,andaman and nicobar islands,2021-04-01,2022-03-01,3207.208333,3562.750000
...,...,...,...,...,...,...,...,...,...,...
211,West Bengal,"Financial Year (Apr - Mar), 2018",437264.333333,228060.375000,147527.666667,west bengal,2018-04-01,2019-03-01,375588.041667,437264.333333
212,West Bengal,"Financial Year (Apr - Mar), 2019",492672.916667,247154.375000,164122.333333,west bengal,2019-04-01,2020-03-01,411276.708333,492672.916667
213,West Bengal,"Financial Year (Apr - Mar), 2020",511896.625000,213856.750000,195039.666667,west bengal,2020-04-01,2021-03-01,408896.416667,511896.625000
214,West Bengal,"Financial Year (Apr - Mar), 2021",455467.500000,249005.666667,155372.583333,west bengal,2021-04-01,2022-03-01,404378.250000,455467.500000


In [220]:
df_per_year["state"] = df_per_year.apply(lambda x : match_state_name(x["State"],unique_state_list),axis=1)
df_per_year["start_date"] = df_per_year.apply(lambda x : get_start_date(x["Year"]),axis=1)
df_per_year["end_date"] = df_per_year.apply(lambda x : get_end_date(x["Year"]),axis=1)
df_per_year["registered_payer"] = df_per_year.apply(lambda x : x["Gst ( goods and service tax ) payers registered before due date_avg"] +x["Gst ( goods and service tax ) payers registered after due date_avg"] ,axis=1) 
df_per_year["eligible_payer"] = df_per_year.apply(lambda x : x["Payer eligible for gst ( goods and service tax ) registration_avg"],axis=1)
df_per_year["return_type"] = df_per_year.apply(lambda x : x["Gst ( goods and service tax ) return type"],axis=1)


KeyError: 'Gst ( goods and service tax ) return type'

In [221]:
df_per_month["state"] = df_per_month.apply(lambda x : match_state_name(x["State"],unique_state_list),axis=1)
df_per_month["date"] = df_per_month.apply(lambda x : get_month(x["Month"]),axis=1)
df_per_month["registered_payer"] = df_per_month.apply(lambda x : x["Gst ( goods and service tax ) payers registered before due date_avg"] +x["Gst ( goods and service tax ) payers registered after due date_avg"] ,axis=1) 
df_per_month["eligible_payer"] = df_per_month.apply(lambda x : x["Payer eligible for gst ( goods and service tax ) registration_avg"],axis=1)
df_per_month["return_type"] = df_per_month.apply(lambda x : x["Gst ( goods and service tax ) return type"],axis=1)



In [ ]:
df_per_year_transformed = df_per_year[["State","Start Date","End Date","Payer Registered for GST","Payer Elibigle for GST"]]

In [224]:
df_per_month_transformed =  df_per_month[["state","return_type","date","registered_payer","eligible_payer"]]

In [ ]:
df_per_year_transformed.dropna()

,state,return_type,date,registered_payer,eligible_payer
0,chandigarh,GSTR-3,2021-02-01,12708.0,12710.0
1,sikkim,GSTR-1,2019-12-01,6056.0,7450.0
2,sikkim,GSTR-3,2019-12-01,6486.0,7450.0
3,sikkim,GSTR-1,2020-01-01,2761.0,3908.0
4,jammu and kashmir,GSTR-1,2017-07-01,45971.0,52455.0
...,...,...,...,...,...
4243,the dadra and nagar haveli and daman and diu,GSTR-3,2022-03-01,13594.0,14581.0
4244,the dadra and nagar haveli and daman and diu,GSTR-1,2022-04-01,10632.0,11569.0
4245,the dadra and nagar haveli and daman and diu,GSTR-3,2022-04-01,10572.0,11569.0
4246,the dadra and nagar haveli and daman and diu,GSTR-1,2022-05-01,10308.0,11640.0


In [226]:
df_per_year_transformed.dropna()

,state,start_date,end_date,registered_payer,eligible_payer
0,andaman and nicobar islands,2017-04-01,2018-03-01,2596.500000,3388.833333
1,andaman and nicobar islands,2018-04-01,2019-03-01,2871.958333,4107.000000
2,andaman and nicobar islands,2019-04-01,2020-03-01,2914.541667,3570.041667
3,andaman and nicobar islands,2020-04-01,2021-03-01,3049.166667,3627.708333
4,andaman and nicobar islands,2021-04-01,2022-03-01,3207.208333,3562.750000
...,...,...,...,...,...
211,west bengal,2018-04-01,2019-03-01,375588.041667,437264.333333
212,west bengal,2019-04-01,2020-03-01,411276.708333,492672.916667
213,west bengal,2020-04-01,2021-03-01,408896.416667,511896.625000
214,west bengal,2021-04-01,2022-03-01,404378.250000,455467.500000


In [227]:
df_per_year_transformed.to_csv("trans_register_payers_per_year_data.csv")
df_per_month_transformed.to_csv("trans_register_payers_per_month_data.csv")